# Tarea 2

**IMPORTANTE: En esta tarea debe usar las bases de datos 'movies' y 'ratings' que se encuentran en la carpeta adjunta. En esta carpeta también encontrará otras bases de datos (que no son necesarias para la tarea) y el README. El README describe cómo están organizadas las bases de datos, qué información tienen, cuántas filas tienen, cómo se relacionan las unas con las otras, etc. Se recomienda encarecidamente leerlo.**

---

Un productor de cine está interesado en producir una película. Como quiere obtener ganancias de esta producción, recurre a usted para que le asesore en la elección de temática de la película.

Usted dispone de una base de datos creada a partir de la información de $610$ usuarios. Cada uno de ellos dio su opinión (de 0.5 a 5 estrellas) de por lo menos $20$ de un total de $9742$ películas.

Como no desea que su asesoría dañe a la privacidad de sus usuarios, responderá cada una de las preguntas del productor de manera diferencialmente-privada.

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
#importar base de datos entregada
from google.colab import drive
drive.mount('/content/drive')

movies = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/4.a Analisis Diferencial/ml-latest-small/movies.csv')
movies.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [36]:
ratings = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/4.a Analisis Diferencial/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [37]:
rng = np.random.default_rng()

# Primera parte (45%)

El productor piensa que la forma más probable de obtener ganancias de la película es que esta pertenezca al género mejor evaluado por los usuarios.

**Problema 1: Determine, usando Report Noisy Max con 0.5-DP, cuál de todos los géneros es el mejor evaluado y cuál es su puntaje.**

**Nota: Recuerde que para poder implementar Report Noisy Max debe hallar una cota para la sensibilidad de las consultas de las que quiere extraer el máximo.**

**Es importante notar que a cada usuario le corresponden varias filas de la base de datos 'ratings'.**


---



Entenderemos el puntaje dado por una persona $i$ a un género como el promedio de los puntajes de las películas que valoró la persona $i$ que pertenecen al género.

Por ejemplo, si las únicas películas que la persona $i$ vio del género 'Aventura' son: 'Toy Story', 'Tarzán' y 'El Rey León', entonces:

$$\text{Puntaje}_i \text{ Aventura}= \frac{\text{Puntaje}_i \text{ Toy Story}+\text{Puntaje}_i \text{ Tarzán}+\text{Puntaje}_i \text{ El Rey León}}{3}$$

Note que una persona puede dar puntaje $0$ a un género (por ejemplo, si no ha visto ninguna película perteneciente a ese género).

---

Entenderemos el puntaje de un determinado género como el promedio de los puntajes de ese género de todos los usuarios de la base de datos 'ratings' (son 610 usuarios en total).

Por ejemplo, el puntaje del género 'Aventura' será:

$$\text{Puntaje Aventura} = \frac{\text{Puntaje}_1 \text{ Aventura}+\text{Puntaje}_2 \text{ Aventura}+\ldots+\text{Puntaje}_{610} \text{ Aventura}}{610},$$
donde $\text{Puntaje}_i \text{ Aventura}$ es el puntaje que le dio la persona $i$ al género aventura.

In [38]:
genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

### Respuesta

In [39]:
#Cruzamos la base de datos de ratings con la de movies para traernos el género 

movies_genres = ratings.merge(movies[['movieId', 'genres', 'title']], on='movieId', how='left')
movies_genres

,userId,movieId,rating,timestamp,genres,title
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,1,3,4.0,964981247,Comedy|Romance,Grumpier Old Men (1995)
2,1,6,4.0,964982224,Action|Crime|Thriller,Heat (1995)
3,1,47,5.0,964983815,Mystery|Thriller,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,964982931,Crime|Mystery|Thriller,"Usual Suspects, The (1995)"
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Drama|Horror|Thriller,Split (2017)
100832,610,168248,5.0,1493850091,Action|Crime|Thriller,John Wick: Chapter Two (2017)
100833,610,168250,5.0,1494273047,Horror,Get Out (2017)
100834,610,168252,5.0,1493846352,Action|Sci-Fi,Logan (2017)


In [40]:
# Primero, crearemos una función la cual nos devuelve el puntaje de cada género. 

def calculate_genre_rating(df, genre):
  df = df.assign(genres=df['genres'].str.split('|')).explode('genres')
  df = df[df['genres'] == (genre)] #Filtra los registros del género que se está evaluando
  user_avg_rating = df.groupby(['userId'])['rating'].mean().reset_index() #Calcula el puntaje para cada persona que evaluó alguna película del género
  rating = user_avg_rating['rating'].sum() / 610 

  return rating


In [41]:
calculate_genre_rating(movies_genres, 'Action')

3.544853882399624

Ahora, creamos la función Report Noisy Max para encontrar el género mejor evaluado. Esta función tendrá como parámetros a x, quien corresponde al df de entrada; R que corresponde a los géneros a evaluar, u que corresponde a la función o fórmula para calcular el mejor género. En este caso, corresponde a la función que creamos previamente. Y finalmente, la sensibilidad y el epsilon.

In [42]:
def report_noisy_max_genre(x, R, u, sensitivity, epsilon):
  # Calculamos el puntaje para cada elemento de R (Lista de géneros) 
  scores = [u(x, r) for r in R]

  # Agregamos ruido a cada score
  noisy_scores = np.add(scores, rng.laplace(scale=sensitivity/epsilon, size=len(scores)))

  # Encontramos el ìndice del máximo
  max_idx = np.argmax(noisy_scores)

  # Retormamos el elemento correspondiente a ese ìndice
  return [R[max_idx], noisy_scores[max_idx]]


**Sensibilidad**

Para este ejercicio, la cota de sensibilidad máxima viene dada por **5/610** la cual corresponde a la máxima diferencia entre la base de datos S y una base de datos vecina S'. La máxima diferencia en calificación que puede existir para una persona es 5, que es la diferencia entre la mayor calificación posible (5) menos la menor calificación posible (0). Esto dividido por el número de registros que tien la base de datos S.


**Resultado**

In [43]:
report_noisy_max_genre(movies_genres, genres , calculate_genre_rating, 5/610, 0.5)


['Drama', 3.7393162929491917]

### CONCLUSIÓN

El género con mayor puntaje es Drama, con una puntaje 0.5-DP de 3.739.

# Segunda parte (45%)

El productor ahora quiere saber cuál de las películas del género mejor evaluado es la que tiene el mayor puntaje, de esta manera puede tener una mejor idea de qué temáticas son las más valoradas por la audiencia.

Usted, sin embargo, se percata de que dentro de las películas del género mejor evaluado hay algunas que tienen muy pocas valoraciones hechas por usuarios. Por lo que su consulta es altamente sensible.

Así que usted, en lugar de responderle cuál es la película con mayor puntaje dentro de todas aquellas que pertenecen al género mejor evaluado, responderá privadamente cuál de todas las películas que tienen más de $100$ valoraciones y que pertenecen al género mejor evaluado es la de puntaje más alto.

---

**Problema 2: Determine, usando Report Noisy Max con 2-DP, cuál de todas las películas del género mejor evaluado que también poseen más de 100 valoraciones hechas por usuarios es la de mayor puntaje.**

**Nota: Recuerde que para poder implementar Report Noisy Max debe hallar una cota para la sensibilidad de las consultas de las que quiere extraer el máximo.**

**Es importante notar que a cada usuario le corresponden muchas filas y no solo una.**

Entenderemos el puntaje de una determinada película como el promedio de los puntajes dados por todos los usuarios que valoraron esa película.

Por ejemplo, si la  película es Toy Story y los usuarios $z_1,\ldots,z_n$ emitieron algún veredicto acerca de Toy Story, entonces:

$$\text{Puntaje Toy Story} = \frac{\text{puntaje}_1 \text{Toy Story}+\ldots +\text{puntaje}_n \text{Toy Story}}{n},$$
donde $\text{puntaje}_1 \text{Toy Story}$ es el puntaje (de 0.5 a 5) que le dio el usuario $z_1$, $\text{puntaje}_2 \text{Toy Story}$ es el puntaje que le dio el usuario $z_2$ y así.

**IMPORTANTE:** Si un usuario no emitió ninguna valoración acerca de la película, entonces este no es considerado en el cálculo del promedio.

---



**Respuesta:**

Primero, creamos la función que calcula el puntaje de una determinada película como el promedio de los puntajes dados por todos los usuarios que valoraron esa película:

In [45]:
def calculate_movie_rating(df, movie):
  df2 = df.assign(genres=df['genres'].str.split('|')).explode('genres')
  df2 = df2[df2['title'] == (movie)] #Filtra los registros del género que se está evaluando
  avg_rating = df2.groupby(['movieId'])['rating'].mean()
  return avg_rating.iloc[0]

In [46]:
calculate_movie_rating(movies_genres, 'Ghost (1990)')

3.4347826086956523

Ahora, crearemos la función Report Noise Max para entregar la película con el mayor puntaje, que pertenece al género mejor evaluado y que tiene más de 100 valoraciones.

La función será similar a la desarrollada en el punto 1, pero en lugar de recibir como input una lista de todos los géneros, recibirá como argumento el género mejor evaluado, el cual se calculará utilizando la función `report_noisy_max_genre` del punto 1.

In [58]:
def report_noisy_max_movie(x, R, u, sensitivity, epsilon): 
  df2 = x.assign(genres=x['genres'].str.split('|')).explode('genres')
  df2 = df2[df2['genres'] == R]
  df2 = df2[df2.groupby('movieId')['rating'].transform('size') > 100] # Dataframe con las películas que pertenecen al género mejor evaluado y que tienen >100 valoraciones
  unique_movie_title = df2['title'].unique().tolist() # Creamos una lista de los movieId de dichas películas para calcular su puntaje y luego agregar el ruido correspondiente.

  # Calculamos el puntaje para cada elemento de R (Lista de películas que cumplen con el criterio) 
  scores_2 = [u(df2, r) for r in unique_movie_title]

  # Agregamos ruido a cada score
  noisy_scores_2 = np.add(scores_2, rng.laplace(scale=sensitivity/epsilon, size=len(scores_2)))

  # Encontramos el ìndice del máximo
  max_idx = np.argmax(noisy_scores_2)

  return [unique_movie_title[max_idx], noisy_scores_2[max_idx]]

In [59]:
report_noisy_max_movie(movies_genres, report_noisy_max_genre(movies_genres, genres , calculate_genre_rating, 5/610, 0.5)[0], calculate_movie_rating, 5/610, 2)

['Shawshank Redemption, The (1994)', 4.427430748880005]

### Conclusión

La película mejor calificada que corresponde al género mejor calificado y que tiene más de 100 calificaciones es Shawshank Redemption, The (1994), con una calificación 2-DP igual a 4.4274.

# Tercera parte (10%)

**Problema 3: Determine el parámetro $\varepsilon$ que se paga por liberar la respuesta de la Primera parte junto a la respuesta de la Segunda parte. Justifique su respuesta.**

---

**Nota: Recuerde que la composición simple posee una variante adaptativa:**

Teorema (Composición Simple Adaptativa): Sea $S\in{\cal Z}^{n}$ una base de datos y sean $F_1(S)$, $F_2(S, \cdot)$ dos mecanismos $\varepsilon_1$-DP y $\varepsilon_2$-DP, respectivamente. Luego
$$H(S)=(F_1(S),F_2(S,F_1(S)))$$
es $(\varepsilon_1+\varepsilon_2)$-DP.

---

La diferencia de este tipo de composición con respecto a la composición simple usual es que ahora permitimos que el segundo mecanismo "vea" el resultado del primero. En este sentido, la segunda consulta puede estar *adaptada* a la respuesta de la primera.

### Respuesta

De acuerdo con el teorema de composición adaptativa, Sea $S\in{\cal Z}^{n}$ una base de datos y sean $F_1(S)$, $F_2(S, \cdot)$ dos mecanismos $\varepsilon_1$-DP y $\varepsilon_2$-DP, respectivamente. Luego
$$H(S)=(F_1(S),F_2(S,F_1(S)))$$
es $(\varepsilon_1+\varepsilon_2)$-DP.


Por tanto, como la primera consulta era 0.5-DP y la segunda era 2-DP,  el parámetro $\varepsilon$ que se paga por liberar la respuesta de la Primera parte junto a la respuesta de la Segunda parte es 2.5-DP